<a href="https://colab.research.google.com/github/SaiKamalMakthala/LLM-Hands-on/blob/main/Capstone_SCRI_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Clone the InvAgent repository
!git clone https://github.com/zefang-liu/InvAgent.git
%cd InvAgent

# Install dependencies
!pip install -r requirements.txt


Cloning into 'InvAgent'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 37 (delta 4), reused 4 (delta 4), pack-reused 28 (from 1)
Receiving objects: 100% (37/37), 35.43 KiB | 1.86 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/InvAgent
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install lz4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.5 MB/s eta 0:00:00


In [4]:
# Make sure we're in the correct working directory
import os
import sys

# Add current path and subfolders to sys.path for imports
project_path = os.getcwd()
sys.path.append(project_path)

# Verify structure
!ls


InvAgent  sample_data


In [5]:
# Add the 'src' folder to Python path
import sys
sys.path.append("/content/InvAgent/src")


# Correct imports based on the actual repo structure
from env import InventoryManagementEnv
from baseline import FixedPolicy
from config import env_configs


In [7]:
import numpy as np
import pandas as pd
from config import env_configs
from env import InventoryManagementEnv
from baseline import FixedPolicy

# Choose one scenario config
config = env_configs['normal_demand']

# Initialize the environment
env = InventoryManagementEnv(**config)

# Create policy config for compatibility
policy_config = {'env_config': config}

# Initialize FixedPolicy
policy = FixedPolicy(env.observation_space, env.action_space, policy_config)

# Reset the environment and extract stage_0's state
raw_state = env.reset()
if isinstance(raw_state, tuple):
    raw_state = raw_state[0]

stage_key = 'stage_0'
state = raw_state[stage_key]

# Run simulation
resilience_metrics = []

for step in range(config['num_periods']):
    print(f"\nStep {step} - State: {state}")

    # Ensure correct format
    state_array = np.array(state)

    # Parse the state
    parsed_state = env._parse_state(state_array)

    # Get action using FixedPolicy (this is where we need to modify the action format)
    action_dict = {}
    for stage in range(env.num_stages):  # Ensure we generate an action for each stage
        stage_key = f'stage_{stage}'
        action_dict[stage_key] = policy.get_fixed_inventory_action(parsed_state, policy_name='sale')

    # Take environment step with action dictionary
    next_raw_state, reward, terminated, truncated, info = env.step(action_dict)
    done = terminated or truncated
    resilience_metrics.append(info)

    # Prepare for next step
    if isinstance(next_raw_state, tuple):
        next_raw_state = next_raw_state[0]
    state = next_raw_state[stage_key]

    if done:
        break

# Display result
df = pd.DataFrame(resilience_metrics)
print(df.head())



Step 0 - State: [20  9  8  1  1  1 12  0  0  0  0  0  0  0  0  0  0]
  stage_0 stage_1 stage_2 stage_3
0      {}      {}      {}      {}


In [15]:
import numpy as np
from ray.rllib.env.multi_agent_env import MultiAgentEnv

class SafeMultiAgentEnv(MultiAgentEnv):
    def __init__(self, config):
        super().__init__()
        self.agents = ['agent_0', 'agent_1']  # example
        self.action_space = ...  # your action space
        self.observation_space = ...  # your obs space
        # Any other init logic

    def reset(self):
        self.dones = {agent: False for agent in self.agents}
        obs = {agent: self._get_obs(agent) for agent in self.agents}
        return obs

    def step(self, action_dict):
        obs, rewards, dones, infos = {}, {}, {}, {}

        for agent_id, action in action_dict.items():
            available_actions = self._get_available_actions(agent_id)

            if len(available_actions) == 0:
                # Safe fallback if no actions are available
                chosen_action = self._get_default_action(agent_id)
                print(f"[WARNING] No valid actions for {agent_id}, using fallback: {chosen_action}")
            else:
                # Sample action from available actions based on your policy
                q_values = self._get_q_values(agent_id)  # custom function
                filtered_qs = [q_values[a] for a in available_actions]

                # Safe argmax
                best_index = int(np.argmax(filtered_qs))
                chosen_action = available_actions[best_index]

            # Execute chosen action
            self._apply_action(agent_id, chosen_action)

            # Build next step data
            obs[agent_id] = self._get_obs(agent_id)
            rewards[agent_id] = self._compute_reward(agent_id)
            dones[agent_id] = self._check_done(agent_id)
            infos[agent_id] = {}

        dones["__all__"] = all(self.dones.values())
        return obs, rewards, dones, infos

    def _get_obs(self, agent_id):
        # Implement observation logic
        pass

    def _get_available_actions(self, agent_id):
        # Return a list of valid actions
        return [0, 1, 2]  # Placeholder

    def _get_default_action(self, agent_id):
        # Return a safe fallback action
        return 0  # or random.choice(self.action_space)

    def _get_q_values(self, agent_id):
        # Return mock Q-values for example
        return [1.0, 0.5, 0.8]  # Example

    def _apply_action(self, agent_id, action):
        # Apply the action to your env state
        pass

    def _compute_reward(self, agent_id):
        # Reward logic
        return 0.0

    def _check_done(self, agent_id):
        # Termination logic
        return False


In [16]:
def compute_resilience_index(data):
    total_cost = sum(d['cost'] for d in data)
    total_stockouts = sum(d['stockout'] for d in data)

    # Find recovery time (e.g., time until stockout returns to 0 after disruption)
    disruption_time = 20
    recovery_time = 0
    recovered = False
    for i, d in enumerate(data[disruption_time:], start=disruption_time):
        if d['stockout'] == 0 and not recovered:
            recovery_time = i - disruption_time
            recovered = True
            break
    if not recovered:
        recovery_time = len(data) - disruption_time

    # Resilience index: lower is better
    index = 1 / (0.5 * total_cost + 0.3 * total_stockouts + 0.2 * recovery_time + 1e-6)
    return index, total_cost, total_stockouts, recovery_time

res_index, cost, stockouts, recovery = compute_resilience_index(resilience_data)

print(f"Resilience Index: {res_index:.6f}")
print(f"Total Cost: {cost}, Stockouts: {stockouts}, Recovery Time: {recovery}")


Resilience Index: -0.250000
Total Cost: 0, Stockouts: 0, Recovery Time: -20
